In [1]:
# Cell 1: Cài đặt thư viện
!pip install transformers torch sentence-transformers tqdm -q

In [5]:
# Cell 2: Import và tải mô hình TinyLlama (1.1B) - NHỎ, NHANH, KHÔNG CẦN TOKEN
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import json
import os

# DÙNG MÔ HÌNH NHỎ, MIỄN PHÍ, KHÔNG GATED
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

print("Đang tải tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

print("Đang tải mô hình TinyLlama (1.1B)...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto",
    low_cpu_mem_usage=True
)

# Tạo pipeline sinh văn bản
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.3,
    top_p=0.9,
    do_sample=True,
    repetition_penalty=1.1,
    pad_token_id=tokenizer.eos_token_id  # Fix warning
)

# Mô hình so sánh câu (tiếng Việt)
print("Đang tải mô hình so sánh...")
similarity_model = SentenceTransformer('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base')

print("Tất cả mô hình đã sẵn sàng! Bắt đầu đánh giá...")

Đang tải tokenizer...


c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ADMIN\.cache\huggingface\hub\models--TinyLlama--TinyLlama-1.1B-Chat-v1.0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed

Đang tải mô hình TinyLlama (1.1B)...


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cuda:0


Đang tải mô hình so sánh...


No sentence-transformers model found with name VoVanPhuc/sup-SimCSE-VietNamese-phobert-base. Creating a new one with mean pooling.
c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ADMIN\.cache\huggingface\hub\models--VoVanPhuc--sup-SimCSE-VietNamese-phobert-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-starte

Tất cả mô hình đã sẵn sàng! Bắt đầu đánh giá...


In [6]:
# Cell 3: Đọc file testset
file_path = "test_set_giao_duc.json"

if not os.path.exists(file_path):
    raise FileNotFoundError(f"Không tìm thấy file: {file_path}")

with open(file_path, 'r', encoding='utf-8') as f:
    testset = json.load(f)

print(f"Đã tải {len(testset)} câu hỏi từ testset.")
print(f"Ví dụ câu hỏi đầu tiên: {testset[0]['question'][:100]}...")

Đã tải 384 câu hỏi từ testset.
Ví dụ câu hỏi đầu tiên: Luật Giáo dục 2019 quy định về những nội dung chính nào?...


In [7]:
# Cell 4: Hàm hỗ trợ
def generate_answer(question):
    prompt = f"Hỏi: {question}\nTrả lời ngắn gọn, chính xác theo Luật Giáo dục 2019:"
    try:
        output = generator(prompt, max_new_tokens=256, num_return_sequences=1)[0]['generated_text']
        # Lấy phần sau "Trả lời:"
        if "Trả lời:" in output:
            answer = output.split("Trả lời:")[-1].strip()
        else:
            answer = output[len(prompt):].strip()
        return answer.split("\n\n")[0].strip()
    except Exception as e:
        return f"[LỖI: {str(e)}]"

def compute_similarity(pred, ref):
    try:
        emb1 = similarity_model.encode(pred, convert_to_tensor=True)
        emb2 = similarity_model.encode(ref, convert_to_tensor=True)
        return util.cos_sim(emb1, emb2).item()
    except:
        return 0.0

In [8]:
# Cell 5: Đánh giá
results = []

print("Bắt đầu đánh giá... (có thể mất vài phút)")
for item in tqdm(testset, desc="Đang xử lý"):
    q = item["question"]
    true_ans = item["answer"]
    
    pred_ans = generate_answer(q)
    sim_score = compute_similarity(pred_ans, true_ans)
    
    results.append({
        "id": item["id"],
        "question": q,
        "true_answer": true_ans,
        "predicted_answer": pred_ans,
        "similarity_score": round(sim_score, 4),
        "reference": item.get("reference", ""),
        "topic": item.get("topic", ""),
        "question_type": item.get("question_type", ""),
        "reasoning_level": item.get("reasoning_level", 0)
    })

# Tính điểm trung bình
avg_score = sum(r["similarity_score"] for r in results) / len(results)
print(f"\nHOÀN TẤT! Độ tương đồng trung bình: {avg_score:.4f}")

Bắt đầu đánh giá... (có thể mất vài phút)


Đang xử lý: 100%|██████████| 384/384 [30:45<00:00,  4.81s/it]


HOÀN TẤT! Độ tương đồng trung bình: 0.5925


In [9]:
# Cell 6: Lưu kết quả
output_file = "evaluation_results.json"

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"Đã lưu kết quả vào: {output_file}")
print(f"   → Mở file để xem chi tiết từng câu hỏi!")

Đã lưu kết quả vào: evaluation_results.json
   → Mở file để xem chi tiết từng câu hỏi!


In [10]:
# Cell 7: Xem mẫu
print("\n" + "="*60)
print("XEM 3 KẾT QUẢ MẪU")
print("="*60)

for r in results[:3]:
    print(f"\nID: {r['id']}")
    print(f"Câu hỏi: {r['question']}")
    print(f"Đáp án thật: {r['true_answer'][:120]}{'...' if len(r['true_answer']) > 120 else ''}")
    print(f"Dự đoán: {r['predicted_answer'][:120]}{'...' if len(r['predicted_answer']) > 120 else ''}")
    print(f"Điểm tương đồng: {r['similarity_score']}")
    print("-" * 50)


XEM 3 KẾT QUẢ MẪU

ID: QDC_001
Câu hỏi: Luật Giáo dục 2019 quy định về những nội dung chính nào?
Đáp án thật: Luật Giáo dục 2019 quy định về hệ thống giáo dục quốc dân; cơ sở giáo dục, nhà giáo, người học; quản lý nhà nước về giáo...
Dự đoán: Những nội dung chính là: Các nguyên tắc giáo dục phù hợp với các nguyên tắc thực hiện trong nhiều nguồn kinh tế, kỹ thuậ...
Điểm tương đồng: 0.7024
--------------------------------------------------

ID: QDC_002
Câu hỏi: Mục tiêu giáo dục theo Luật Giáo dục 2019 là gì?
Đáp án thật: Mục tiêu giáo dục nhằm phát triển toàn diện con người Việt Nam có đạo đức, tri thức, văn hóa, sức khỏe, thẩm mỹ và nghề ...
Dự đoán: Đại học và các trường học có thể làm việc nhóm giáo dục khác nhau. Ngoài ra, một số đại học có thể làm việc nhóm giáo dụ...
Điểm tương đồng: 0.4919
--------------------------------------------------

ID: QDC_003
Câu hỏi: Nền giáo dục Việt Nam có những tính chất và nguyên lý cơ bản nào?
Đáp án thật: Nền giáo dục Việt Nam là nền giáo dục xã